In [ ]:
import os
from dotenv import load_dotenv  # pip install python-dotenv
import psycopg2  # pip install psycopg2-binary
from sqlalchemy import create_engine  # pip install sqlalchemy
 
# Charge le fichier .env
load_dotenv()
 
# Variables simples depuis .env
host = os.getenv('DB_HOST')      # ex: localhost
port = os.getenv('DB_PORT')      # ex: 5432
dbname = os.getenv('DB_NAME')    # ex: biblio_test
user = os.getenv('DB_USER')      # ex: postgres
password = os.getenv('DB_PASSWORD')  # ton mot de passe
 
print("Variables chargées depuis .env")
print(f"User est connecté : {user}")
print(f"Connexion à : {host}:{port}/{dbname}")

Variables chargées depuis .env
User est connecté : postgres
Connexion à : localhost:5432/bibliothequeuniv


In [45]:
try:
    # Crée la connexion psycopg2 (adaptateur direct)
    connexion = psycopg2.connect(
        host=host,
        port=port,
        dbname=dbname,
        user=user,
        password=password
    )
    
    # Cursor = "pointeur" pour exécuter SQL
    curseur = connexion.cursor()
    
    # Test simple : version PostgreSQL
    curseur.execute("SELECT version();")
    resultat = curseur.fetchone()
    print("SUCCESS")
    print(f"PostgreSQL : {resultat[0][:10]}")
    
    # Test table biblio
    curseur.execute("SELECT COUNT(*) FROM etudiants;")
    nb_etudiants = curseur.fetchone()[0]
    print(f"{nb_etudiants} étudiants chargés")
    
    # Ferme proprement
    curseur.close()
    connexion.close()
    
except psycopg2.OperationalError as erreur:
    print("Erreur connexion : vérifie .env/PostgreSQL")
    print(erreur)
except Exception as e:
    print(f"Erreur : {e}")

SUCCESS
PostgreSQL : PostgreSQL
26 étudiants chargés


In [46]:
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
from sqlalchemy import Column, Integer, String, Date, Float, ForeignKey
 
# Base pour modèles ORM
Base = declarative_base()
 
# Modèle Etudiant (comme ta table)
class Etudiant(Base):
    __tablename__ = 'etudiants'  # Nom table Postgres
    
    id = Column(Integer, primary_key=True)  # Clé primaire
    nom = Column(String)
    prenom = Column(String)
    email = Column(String)
    date_inscription = Column(Date)
    solde_amende = Column(Float)
 
# Chaîne connexion SQLAlchemy (utilise psycopg2 en backend)
engine = create_engine(f"postgresql://{user}:{password}@{host}:{port}/{dbname}")
 
try:
    # Crée session ORM (comme un "cursor avancé")
    Session = sessionmaker(bind=engine)
    session = Session()
    
    # Test : Charge TOUS les étudiants (ORM)
    etudiants = session.query(Etudiant).all()  # Comme SELECT *
    # print(f" {e.prenom} {e.nom} (ID: {e.id})")
    
    print("ORM -- SUCCESS! Étudiants :")
    for etu in etudiants[:5]:  # Top 5
        print(f"  - {etu.prenom} {etu.nom} (ID: {etu.id}, email : {etu.email})")
    
    # Exemple requête filtrée (paramétrée auto)
    retardataires = session.query(Etudiant).limit(3).all()
    print(f"3 étudiants (exemple filtre) : {len(retardataires)}")
    
    # Ferme session
    session.close()
    
except Exception as e:
    print(f"Erreur ORM : {e}")
finally:
    print("Fin TP !")

ORM -- SUCCESS! Étudiants :
  - Léa Martin (ID: 1, email : lea.martin@univ.fr)
  - Lucas Durand (ID: 2, email : lucas.durand@univ.fr)
  - Emma Bernard (ID: 3, email : emma.bernard@univ.fr)
  - Noah Petit (ID: 4, email : noah.petit@univ.fr)
  - Chloé Robert (ID: 5, email : chloe.robert@univ.fr)
3 étudiants (exemple filtre) : 3
Fin TP !


/tmp/ipykernel_9173/1712087146.py:6: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


In [47]:
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
from sqlalchemy import Column, Integer, String, Date, Float, ForeignKey
 
# Base pour modèles ORM
Base = declarative_base()
 
# Modèle Etudiant (comme ta table)
class Livre(Base):
    __tablename__ = 'livre'  # Nom table Postgres
    
    isbn = Column(Integer, primary_key=True)  # Clé primaire
    titre = Column(String)
    editeur = Column(String)
    annee = Column(Date)
    exemplaires_dispo = Column(Integer)
 
# Chaîne connexion SQLAlchemy (utilise psycopg2 en backend)
engine = create_engine(f"postgresql://{user}:{password}@{host}:{port}/{dbname}")
 
try:
    # Crée session ORM (comme un "cursor avancé")
    Session = sessionmaker(bind=engine)
    session = Session()
    
    # Test : Charge TOUS les étudiants (ORM)
    livres = session.query(Livre).all()  # Comme SELECT *
    # print(f" {e.prenom} {e.nom} (ID: {e.id})")
    
    print("ORM -- SUCCESS! Livres :")
    for livre in livres[:5]:  # Top 5
        print(f"  - {livre.titre} {livre.editeur} (ISBN: {livre.isbn}, année : {livre.annee})")
    
    # Exemple requête filtrée (paramétrée auto)
    top_livres = session.query(Livre).limit(5).all()
    print(f"5 livres (exemple filtre) : {len(top_livres)}")
    
    # Ferme session
    session.close()
    
except Exception as e:
    print(f"Erreur ORM : {e}")
finally:
    print("Fin TP !")

ORM -- SUCCESS! Livres :
  - Web Development with Node.js Pearson (ISBN: 9780133350680, année : 2019)
  - Fluent Python O'Reilly (ISBN: 9781492045526, année : 2022)
  - Clean Code Prentice Hall (ISBN: 9780132350884, année : 2008)
  - Learning React O'Reilly (ISBN: 9781491950357, année : 2020)
  - Clean Architecture Pearson (ISBN: 9780134494166, année : 2017)
5 livres (exemple filtre) : 5
Fin TP !


/tmp/ipykernel_9173/2887475677.py:6: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


In [2]:
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker, relationship, DeclarativeBase
from sqlalchemy import Column, Integer, String, Date, Float, ForeignKey
from datetime import date, datetime

# Base pour modèles ORM
Base = declarative_base()
 
# Modèle Etudiant (comme ta table)
class Etudiant(Base):
    __tablename__ = 'etudiants'  # Nom table Postgres
    
    id = Column(Integer, primary_key=True)  # Clé primaire
    nom = Column(String)
    prenom = Column(String)
    email = Column(String)
    date_inscription = Column(Date)
    solde_amende = Column(Float)

    emprunt= relationship("Emprunt", back_populates="etudiant", cascade="delete")

# Modèle Livre
class Livre(Base):
    __tablename__ = 'livre'  # Nom table Postgres
    
    isbn = Column(Integer, primary_key=True)  # Clé primaire
    titre = Column(String)
    editeur = Column(String)
    annee = Column(Date)
    exemplaires_dispo = Column(Integer)

    emprunt= relationship("Emprunt", back_populates="livre", cascade="delete")

# Modèle Emprunt
class Emprunt(Base):
    __tablename__ = 'emprunt'  # Doit correspondre au nom réel de la table
    
    id_emprunt = Column(Integer, primary_key=True)  # Clé primaire
    date_emprunt = Column(Date, default=date.today) # format 2026-01-01
    date_retour = Column(Date)
    amende = Column(Float, default=0.0)

    id_etud = Column(Integer, ForeignKey('etudiants.id'))
    isbn = Column(Integer, ForeignKey('livre.isbn'))

    etudiant= relationship("Etudiant", back_populates="emprunt")
    livre= relationship("Livre", back_populates="emprunt")
 
# Chaîne connexion SQLAlchemy (utilise psycopg2 en backend)
engine = create_engine(f"postgresql://{user}:{password}@{host}:{port}/{dbname}")

try:
    # Crée session ORM (comme un "cursor avancé")
    Session = sessionmaker(bind=engine)
    session = Session()
    
    # Test : Charge TOUS les étudiants (ORM)
    emprunts = session.query(Emprunt).all()  # Comme SELECT *
    # print(f" {e.prenom} {e.nom} (ID: {e.id})")
    
    print("ORM -- SUCCESS! Livres :")
    for livre in livres[:5]:  # Top 5
        print(f"  - {livre.titre} {livre.editeur} (ISBN: {livre.isbn}, année : {livre.annee})")
    
    # Exemple requête filtrée (paramétrée auto)
    top_livres = session.query(Livre).limit(5).all()
    print(f"5 livres (exemple filtre) : {len(top_livres)}")
    
    # Ferme session
    session.close()
    
except Exception as e:
    print(f"Erreur ORM : {e}")
finally:
    print("Fin TP !")


Erreur ORM : (psycopg2.errors.UndefinedColumn) column livre.auteur does not exist
LINE 1: ...e.isbn AS livre_isbn, livre.titre AS livre_titre, livre.aute...
                                                             ^
HINT:  Perhaps you meant to reference the column "livre.editeur".

[SQL: SELECT livre.isbn AS livre_isbn, livre.titre AS livre_titre, livre.auteur AS livre_auteur, livre.editeur AS livre_editeur, livre.annee_publication AS livre_annee_publication, livre.exemplaires_dispo AS livre_exemplaires_dispo 
FROM livre]
(Background on this error at: https://sqlalche.me/e/20/f405)
Fin TP !


/tmp/ipykernel_17237/3709642062.py:7: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


In [58]:
def create_etu(session, nom, prenom, email, date_inscription, solde_amende):
    etu = Etudiant(nom=nom,
                   prenom=prenom,
                   email=email,
                   date_inscription=date_inscription,
                   solde_amende=solde_amende)
    session.add(etu)
    session.commit()
    return etu.id

def read_etu(session, nom):
    print(f"Recherche de l'étudiant avec le nom : {nom}")
    return session.query(Etudiant).filter_by(nom=nom).first()

def update_etu(session, id_etud, nouveau_prenom, nouveau_solde):
    etu = session.query(Etudiant).get(id_etud)
    if etu:
        etu.prenom = nouveau_prenom
        etu.solde_amende = nouveau_solde
        session.commit()
        print(f"Étudiant ID={id_etud} mis à jour : prénom={nouveau_prenom}, solde_amende={nouveau_solde}")
        return True
    return False

def delete_etu(session, nom):
    etu = read_etu(session, nom)
    if etu:
        session.delete(etu)
        session.commit()
        return True
    return False

update_etu(session, '26', 'Alexandre', 0)
read_etu(session, 'Gouraud')

Étudiant ID=26 mis à jour : prénom=Alexandre, solde_amende=0
Recherche de l'étudiant avec le nom : Gouraud


/tmp/ipykernel_9173/3535801455.py:16: LegacyAPIWarning: The Query.get() method is considered legacy as of the 1.x series of SQLAlchemy and becomes a legacy construct in 2.0. The method is now available as Session.get() (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  etu = session.query(Etudiant).get(id_etud)


In [61]:
# isbn = Column(Integer, primary_key=True)  # Clé primaire
# titre = Column(String)
# editeur = Column(String)
# annee = Column(Date)
# exemplaires_dispo = Column(Integer)

def create_livre(session, isbn, titre, editeur, annee, exemplaires_dispo):
    livre = Livre(isbn=isbn,
                  titre=titre,
                  editeur=editeur,
                  annee=annee,
                  exemplaires_dispo=exemplaires_dispo)
    session.add(livre)
    session.commit()
    return livre.isbn

def read_livre(session, isbn):
    print(f"Recherche du livre avec l'ISBN : {isbn}")
    return session.query(Livre).filter_by(isbn=isbn).first()

def update_livre(session, isbn, nouveau_titre, nouveau_exemplaires_dispo):
    livre = session.query(Livre).get(isbn)
    if livre:
        livre.titre = nouveau_titre
        livre.exemplaires_dispo = nouveau_exemplaires_dispo
        session.commit()
        print(f"Livre ISBN={isbn} mis à jour : titre={nouveau_titre}, exemplaires_dispo={nouveau_exemplaires_dispo}")
        return True
    return False

def delete_livre(session, isbn):
    livre = read_livre(session, isbn)
    if livre:
        session.delete(livre)
        session.commit()
        return True
    return False

read_livre(session, '10')

Recherche du livre avec l'ISBN : 10


In [ ]:
def menu_crud():
    print("\n=== MENU CRUD ETUDIANT ===")
    print("1: Créer")
    print("2: Lire")
    print("3: Lire par ID")
    print("4: Update")
    print("5: Delete")
    print("0: Quitter")
    return input("Choix (0-5): ")
 
# BOUCLE PRINCIPALE
while True:
    choix = menu_crud()
    
    if choix == "1":  # CREATE
        nom = input("Nom: ").upper()
        prenom = input("Prénom: ")
        id_new = create_etu(session, nom, prenom)
        print(f"Créé ID={id_new}")
    
    elif choix == "2":  # READ ALL
    # session.query(Etudiant).all()
        nom = input("Nom à chercher: ").upper()
        e = read_etu(session, nom)
        if e:
            print(f" {e.prenom} {e.nom} (ID={e.id})")
        else:
            print(" Non trouvé")
            
    elif choix == "3": # Read by ID
        id = int(input("ID à chercher: "))
        e = session.query(Etudiant).get(id)
        if e:
            print(f" {e.prenom} {e.nom} (ID={e.id})")
        else:
            print(" Non trouvé")
    
    elif choix == "4":  # UPDATE
        id = int(input("ID de l'utilisateur à modifier: "))
        e = session.query(Etudiant).get(id)
        new_prenom = input("Nouveau prénom: ")
        if update_etu(session, id, new_prenom):
            print(f" Prénom modifié {new_prenom} de l'ID {id} prénom {e.prenom} ")
        else:
            print("ID inexistant")
            
            
    elif choix == "5": # Update Solde Amende
        id = int(input("ID de l'utilisateur à modifier: "))
        e = session.query(Etudiant).get(id)
        new_solde = float(input("Nouveau solde amende: "))
        if update_etu(session, id, new_solde):
            print(f" Solde amende modifié {new_solde} de l'ID {id} prénom {e.amende} ")
        else:
            print("ID inexistant")
    
    elif choix == "7":  # DELETE
        nom = input("Nom à supprimer: ").upper()
        if delete_etu(session, nom):
            print("Supprimé")
        else:
            print(" Non trouvé")
    
    elif choix == "0": # Fermeture du Code
        print("Au revoir !")
        break
    
    else:
        print(" Choix invalide")
 
session.close()


=== MENU CRUD ETUDIANT ===
1: Créer
2: Lire
3: Lire par ID
4: Update
5: Delete
0: Quitter
 Choix invalide

=== MENU CRUD ETUDIANT ===
1: Créer
2: Lire
3: Lire par ID
4: Update
5: Delete
0: Quitter
Au revoir !


In [64]:
def menu_crud():
    print("\n=== MENU CRUD LIVRE ===")
    print("1: Créer")
    print("2: Lire")
    print("3: Lire par ID")
    print("4: Update")
    print("5: Delete")
    print("0: Quitter")
    return input("Choix (0-5): ")
 
# BOUCLE PRINCIPALE
while True:
    choix = menu_crud()
    
    if choix == "1":  # CREATE
        isbn = input("ISBN: ")
        titre = input("Titre: ")
        editeur = input("Éditeur: ")
        annee = input("Année: ")
        exemplaires_dispo = int(input("Exemplaires disponibles: "))
        id_new = create_livre(session, isbn, titre, editeur, annee, exemplaires_dispo)
        print(f"Créé ISBN={id_new}")
    
    elif choix == "2":  # READ ALL
        livres = session.query(Livre).all()  # Comme SELECT *
        for livre in livres:
            print(f"{livre.titre} {livre.editeur} (ISBN: {livre.isbn}, année : {livre.annee})")
        
            
    elif choix == "3": # Read by ID
        isbn = input("ISBN à chercher: ")
        l = session.query(Livre).get(isbn)
        if l:
            print(f" {l.titre} {l.editeur} (ISBN={l.isbn})")
        else:
            print(" Non trouvé")
    
    elif choix == "4":  # UPDATE
        isbn = input("ISBN du livre à modifier: ")
        l = session.query(Livre).get(isbn)
        new_titre = input("Nouveau titre: ")
        new_exemplaires_dispo = int(input("Nouveaux exemplaires disponibles: "))
        if update_livre(session, isbn, new_titre, new_exemplaires_dispo):
            print(f"Livre ISBN={isbn} mis à jour : titre={new_titre}, exemplaires_dispo={new_exemplaires_dispo}")
        else:
            print("ID inexistant")
            
            
    elif choix == "5": # Update Solde Amende
        isbn = input("ISBN du livre à modifier: ")
        l = session.query(Livre).get(isbn)
        new_exemplaires_dispo = int(input("Nouveaux exemplaires disponibles: "))
        if update_livre(session, isbn, l.titre, new_exemplaires_dispo):
            print(f"Livre ISBN={isbn} mis à jour : titre={l.titre}, exemplaires_dispo={new_exemplaires_dispo}")
        else:
            print("ID inexistant")
    
    elif choix == "7":  # DELETE
        nom = input("Nom à supprimer: ").upper()
        if delete_etu(session, nom):
            print("Supprimé")
        else:
            print(" Non trouvé")
    
    elif choix == "0": # Fermeture du Code
        print("Au revoir !")
        break
    
    else:
        print(" Choix invalide")
 
session.close()


=== MENU CRUD LIVRE ===
1: Créer
2: Lire
3: Lire par ID
4: Update
5: Delete
0: Quitter
Web Development with Node.js Pearson (ISBN: 9780133350680, année : 2019)
Fluent Python O'Reilly (ISBN: 9781492045526, année : 2022)
Clean Code Prentice Hall (ISBN: 9780132350884, année : 2008)
Learning React O'Reilly (ISBN: 9781491950357, année : 2020)
Clean Architecture Pearson (ISBN: 9780134494166, année : 2017)
Effective Java Addison-Wesley (ISBN: 9780134685991, année : 2018)
Designing Data-Intensive Applications O'Reilly (ISBN: 9781492078005, année : 2017)
The C Programming Language Prentice Hall (ISBN: 9780131103627, année : 1988)
Introduction to Algorithms MIT Press (ISBN: 9780262033848, année : 2009)
Operating System Concepts Wiley (ISBN: 9780131101630, année : 2018)
Computer Networks Pearson (ISBN: 9780135974445, année : 2021)
Data Structures in Java Pearson (ISBN: 9780131873254, année : 2014)
Python Crash Course No Starch Press (ISBN: 9780134853987, année : 2019)
You Don’t Know JS O'Reilly 

In [1]:
import os
from dotenv import load_dotenv  # pip install python-dotenv
import psycopg2  # pip install psycopg2-binary
from sqlalchemy import create_engine  # pip install sqlalchemy
 
# Charge le fichier .env
load_dotenv()
 
# Variables simples depuis .env
host = os.getenv('DB_HOST')      # ex: localhost
port = os.getenv('DB_PORT')      # ex: 5432
dbname = os.getenv('DB_NAME')    # ex: biblio_test
user = os.getenv('DB_USER')      # ex: postgres
password = os.getenv('DB_PASSWORD')  # ton mot de passe
 
print("Variables chargées depuis .env")
print(f"User est connecté : {user}")
print(f"Connexion à : {host}:{port}/{dbname}")

Variables chargées depuis .env
User est connecté : postgres
Connexion à : localhost:5432/bibliothequeuniv
